In [33]:
import pandas as pd
import numpy as np
import requests
from dotenv import load_dotenv
import os
import json
import urllib3


load_dotenv()


True

In [34]:
# Disable SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class DrupalJSONAPIClient:
    def __init__(self, base_url, username, password):
        self.base_url = base_url.rstrip('/')
        self.username = username
        self.password = password
        self.session = requests.Session()
        self.session.verify = False
        self.csrf_token = None
        self.login()
        self.get_csrf_token()

    def login(self):
        """Login to Drupal and get session cookie"""
        login_url = f"{self.base_url}/user/login"
        
        # First get the form build ID and token
        response = self.session.get(login_url)
        
        # Now post the login form
        login_data = {
            'name': self.username,
            'pass': self.password,
            'form_id': 'user_login_form',
            'op': 'Log in'
        }
        
        response = self.session.post(login_url, data=login_data)
        print(f"Login response status: {response.status_code}")
        if response.status_code == 200:
            print("Login successful")
        else:
            print("Login failed")

    def get_csrf_token(self):
        """Get CSRF token from Drupal"""
        token_url = f"{self.base_url}/session/token"
        response = self.session.get(token_url)
        if response.status_code == 200:
            self.csrf_token = response.text
            print(f"Got CSRF token: {self.csrf_token}")
        else:
            print(f"Failed to get CSRF token. Status: {response.status_code}")
        return self.csrf_token

    def create_project(self, project_data):
        url = f"{self.base_url}/jsonapi/node/projects"
        
        headers = {
            'Content-Type': 'application/vnd.api+json',
            'Accept': 'application/vnd.api+json',
            'X-CSRF-Token': self.csrf_token
        }

        # Format the data according to JSON:API specification
        payload = {
            "data": {
                "type": "node--projects",
                "attributes": {
                    "title": project_data.get("project_name", ""),
                    "field_adresse": project_data.get("adresse", ""),
                    "field_description": project_data.get("description", ""),
                    "field_federal_state": project_data.get("federal_state", ""),
                    "field_geo_data": project_data.get("geo_data", ""),
                    "field_hospital": project_data.get("hospital", ""),
                    "field_latitude": project_data.get("latitude", 0),
                    "field_longitude": project_data.get("longitude", 0),
                    "field_project_name": project_data.get("project_name", ""),
                    "field_status": project_data.get("status", "")
                }
            }
        }

        print(f"Making request to: {url}")
        print(f"With headers: {headers}")
        print(f"With payload: {json.dumps(payload, indent=2)}")
        print(f"Session cookies: {self.session.cookies.get_dict()}")
        
        response = self.session.post(url, headers=headers, json=payload)
        
        print(f"Response status code: {response.status_code}")
        print(f"Response headers: {dict(response.headers)}")
        try:
            print(f"Response body: {json.dumps(response.json(), indent=2)}")
        except:
            print(f"Raw response text: {response.text}")
                
        return response

In [2]:
DRUPAL_URL = os.getenv("DRUPAL_URL")
USERNAME = os.getenv("USERNAME")
PASSWORD =  os.getenv("PASSWORD")

In [49]:
ki_a_data = pd.read_excel('Projektliste_KI-Atlas.xlsx')

art_filter = [x for x in ki_a_data["Art"].unique().tolist() if x not in ["", "nan", "Telemedizin",np.nan]]
art_filter

ki_a_data = ki_a_data[ki_a_data["Art"].isin(art_filter)]
#ki_a_data = ki_a_data[ki_a_data["Krankenhaus"] == "Salzburger Landeskliniken (SALK)"]
#ki_a_data

c:\Users\gabriel.schachinger\.conda\envs\GSchachEV_1\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [51]:
def send_project_data_to_drupal(ki_a_data):
    from requests.exceptions import RequestException

    # Create client instance
    try:
        client = DrupalJSONAPIClient(DRUPAL_URL, USERNAME, PASSWORD)
    except Exception as e:
        print(f"Error initializing Drupal client: {e}")
        return

    for index, row in ki_a_data.iterrows():
        try:
            # Sicherstellen, dass alle Werte vorhanden sind
            if pd.isna(row["longitude"]) or pd.isna(row["latitude"]):
                print(f"Skipping row {index}: Missing geolocation data")
                continue

            project_data = {
                "project_name": row.get("Projektname", "Unknown"),
                "adresse": row.get("Adresse", "Unknown"),
                "description": row.get("Projektbeschreibung", ""),
                "federal_state": row.get("Bundesland", ""),  # Leerzeichen entfernt
                "geo_data": f"POINT({row['longitude']} {row['latitude']})",
                "hospital": row.get("Krankenhaus", ""),
                "latitude": row["latitude"],
                "longitude": row["longitude"],
                "status": row.get("Status", "Unknown")
            }

            # Create the project
            response = client.create_project(project_data)

            if response.status_code == 201:
                print(f"Project {project_data['project_name']} created successfully!")
            else:
                print(f"Error creating project {project_data['project_name']}: {response.status_code} - {response.text}")

        except KeyError as ke:
            print(f"Missing expected column in row {index}: {ke}")
        except RequestException as re:
            print(f"Network error while sending project {row.get('Projektname', 'Unknown')}: {re}")
        except Exception as e:
            print(f"Unexpected error processing row {index}: {e}")


In [52]:
send_project_data_to_drupal(ki_a_data)

Login response status: 200
Login successful
Got CSRF token: k9sLPedbLVUos-Ky8jVORGqVoztJOFFQLGPg9yQaFLA
Making request to: https://crispy-enigma-jj77r9v6xp453qjx4-80.app.github.dev/jsonapi/node/projects
With headers: {'Content-Type': 'application/vnd.api+json', 'Accept': 'application/vnd.api+json', 'X-CSRF-Token': 'k9sLPedbLVUos-Ky8jVORGqVoztJOFFQLGPg9yQaFLA'}
With payload: {
  "data": {
    "type": "node--projects",
    "attributes": {
      "title": "Spracherkennung mit Dragon1",
      "field_adresse": "Johannes von Gott-Platz 1, 7000 Eisenstadt, Austria",
      "field_description": "Spracherkennung \"Dragon1\" von Nuance die mit einer KI die Sprache erkennt.\nSprachdiktat und Dokumentation um im KIS die Felder zu bef\u00fcllen.",
      "field_federal_state": "",
      "field_geo_data": "POINT(16.513804 47.8476293)",
      "field_hospital": "Barmherzige Br\u00fcder Eisenstadt",
      "field_latitude": 47.8476293,
      "field_longitude": 16.513804,
      "field_project_name": "Sprach